<center><img src="https://github.com/MAGALLANESJoseManuel/OpenDataBigData_at_quality4point0/blob/main/logo.png?raw=true
" width="1000"></center>



# The Dengue and Census: What Province should be a priority?



## ACCESSING

The clean data for dengue:

In [ ]:

GitFolder="https://github.com/MagallanesTalks/OpenBigData_atPUCP/raw/refs/heads/main/data/"
linkDengue=GitFolder + "dengue_ok.parquet"

import pandas as pd

dengue_ok=pd.read_parquet(linkDengue)

Let's compute the population of the more vulnerable people to Dengue:

In [ ]:

sickInfant=((dengue_ok.edad<=2) & (dengue_ok.tipo_edad=='A')) | \
           ((dengue_ok.edad<=48) & (dengue_ok.tipo_edad=='M'))| \
           ((dengue_ok.tipo_edad=='D'))
sickElder=((dengue_ok.edad>=65) & (dengue_ok.tipo_edad=='A'))

dengue_ok=dengue_ok.assign(sick_elder=sickElder)
dengue_ok=dengue_ok.assign(sick_infant=sickInfant)

colsNeeded=['ubigeo','provincia','sick_elder','sick_infant']
dengue_ok=dengue_ok.loc[:,colsNeeded]

colsForGroup=['ubigeo','provincia']
dengue_sick=dengue_ok.groupby(colsForGroup,as_index=False).sum()
dengue_sick=dengue_sick.iloc[:-1,:]

dengue_sick

In [ ]:
dengue_sick.info()

In [ ]:
import altair as alt

# Create the scatter plot
alt.Chart(dengue_sick).mark_circle().encode(
    alt.X('sick_elder:Q'),
    alt.Y('sick_infant:Q'),
    tooltip=['provincia', 'sick_elder', 'sick_infant']
).interactive()



In [ ]:
linkEdad=GitFolder+"edadClean.parquet"

import pandas as pd


edad=pd.read_parquet(linkEdad)

In [ ]:
edad.info()

In [ ]:
edad.columns

Let's create simpler data frame, just with the vulnerable ages:

In [ ]:
edadCenso=edad.iloc[:,:3]
edadCenso['popInfant']=edad.loc[:,'Edad_0':'Edad_2_años'].apply(sum,axis=1)
edadCenso['popElder']=edad.loc[:,'Edad_65_años':].apply(sum,axis=1)
edadCenso

In [ ]:
edadCenso.info()

We should use ubigeos to merge both data:

In [ ]:
dengue_sick.ubigeo

In [ ]:
edadCenso.ubigeo

As you see, the ubigeo needs some changes:

In [ ]:
dengue_sick['ubigeo']=dengue_sick.ubigeo.astype(str)
edadCenso['ubigeo']=edadCenso.ubigeo.astype(str)

In [ ]:
dengue_sick.info()

In [ ]:
edadCenso.info()

In [ ]:
dengueEdad=edadCenso.merge(dengue_sick,on='ubigeo')

#here you are:
dengueEdad

Some extra steps:

In [ ]:
#not needed
dengueEdad.drop(columns=['provincia'],inplace=True)

# computations
dengueEdad['infant_share']=100*dengueEdad.sick_infant/dengueEdad.popInfant
dengueEdad['elder_share']=100*dengueEdad.sick_elder/dengueEdad.popElder

# here
dengueEdad

Discuss:

In [ ]:
# Create the scatter plot
alt.Chart(dengueEdad).mark_circle().encode(
    alt.X('elder_share:Q'),
    alt.Y('infant_share:Q'),
    tooltip=['Provincia', 'elder_share', 'infant_share']
).interactive()

In [53]:
import os

folder = "data"

# Check whether the specified path exists or not
if not os.path.exists(folder):
   # Create a new directory because it does not exist
   os.makedirs(folder)
   dengueEdad.to_csv(os.path.join('data','dengueEdad.csv'),index=False)
   dengueEdad.to_parquet(os.path.join('data','dengueEdad.parquet'),index=False)

else:
    dengueEdad.to_csv(os.path.join('data','dengueEdad.csv'),index=False)
    dengueEdad.to_parquet(os.path.join('data','dengueEdad.parquet'),index=False)